In [1]:
import xarray as xr
import numpy as np
import sys, os, glob
import time
from scipy.stats import weibull_min
import dask.distributed as dd

root_dir = '/media/harish/SSD_4TB/EU_SCORES_project'
scripts_dir = f'{root_dir}/scripts'
sys.path.append(scripts_dir)

from data_processing.libraries import mean_statistics, std_statistics, wind_power_density, weibull , weibull_statistics

In [4]:
client.close()
cluster.close()

In [ ]:
# Create a Dask cluster
print("Starting parallel computing...")
import dask.distributed as dd
cluster = dd.LocalCluster(n_workers=8, threads_per_worker=2,memory_limit='2GB',dashboard_address=':8787')
# Connect to the cluster
client = dd.Client(cluster)
print(client)

In [5]:
root_dir = '/media/harish/SSD_4TB/EU_SCORES_project'
run = 'New_runs'
cases = ['Germany_coast', 'Portugal_coast', 'Ireland_coast', 'Netherlands_coast']
for case in cases:
    run_dir=f'{root_dir}/WRFV4.4/EU_SCORES/{run}/{case}/Postprocessed/variablewise_files'

    file_names = ['ws_10', 'ws_80', 'ws_100', 'ws_120', 'ws_150',
                'wpd_80', 'wpd_100', 'wpd_120', 'wpd_150',
                '8MW/tp_80', '8MW/tp_100', '8MW/tp_120', '8MW/tp_150',
                '15MW/tp_80', '15MW/tp_100', '15MW/tp_120', '15MW/tp_150',
                'spv','SWDOWN2','T2']
    for file_name in file_names:

        print(f'Checking {case} {file_name}')
        with xr.open_dataset(f'{run_dir}/{file_name}.nc') as ds:
            if 'XLAT' not in ds.coords and 'XLONG' not in ds.coords:
                print(f'No lat lon in {case} {file_name}')

Checking Germany_coast ws_10
Checking Germany_coast ws_80
Checking Germany_coast ws_100
Checking Germany_coast ws_120
Checking Germany_coast ws_150
Checking Germany_coast wpd_80
Checking Germany_coast wpd_100
Checking Germany_coast wpd_120
Checking Germany_coast wpd_150
Checking Germany_coast 8MW/tp_80
Checking Germany_coast 8MW/tp_100
Checking Germany_coast 8MW/tp_120
Checking Germany_coast 8MW/tp_150
Checking Germany_coast 15MW/tp_80
Checking Germany_coast 15MW/tp_100
Checking Germany_coast 15MW/tp_120
Checking Germany_coast 15MW/tp_150
Checking Germany_coast spv
Checking Germany_coast SWDOWN2
Checking Germany_coast T2
Checking Portugal_coast ws_10
Checking Portugal_coast ws_80
Checking Portugal_coast ws_100
Checking Portugal_coast ws_120
Checking Portugal_coast ws_150
Checking Portugal_coast wpd_80
Checking Portugal_coast wpd_100
Checking Portugal_coast wpd_120
Checking Portugal_coast wpd_150
Checking Portugal_coast 8MW/tp_80
Checking Portugal_coast 8MW/tp_100
Checking Portugal_coas

In [3]:
def overall_statistic(ds,statistic,time_coord='Time',quantile=None):
    if statistic == 'std':
        return ds.std(dim=time_coord).compute()
    elif statistic == 'quantile':
        return ds.quantile(quantile,dim=time_coord,method='inverted_cdf').compute()
    else:
        print(f'Invalid statistic {statistic}')
        return None

In [2]:
root_dir = '/media/harish/SSD_4TB/EU_SCORES_project'
run = 'New_runs'

chunks={"Time": -1,"south_north": 8,"west_east": 8}
case = 'Portugal_coast'
file_name = 'T2'
variable = 'T2'
run_dir=f'{root_dir}/WRFV4.4/EU_SCORES/{run}/{case}/Postprocessed/variablewise_files'
target_dir=f'{root_dir}/WRFV4.4/EU_SCORES/{run}/{case}/Postprocessed/statistics_files/{file_name}'

statistic = 'std'
ds = xr.open_dataset(f'{target_dir}/{statistic}.nc')
ds

<xarray.Dataset>
Dimensions:         (hour: 24, south_north: 127, west_east: 127, month: 12,
                     year: 31)
Coordinates:
  * hour            (hour) int64 0 1 2 3 4 5 6 7 8 ... 16 17 18 19 20 21 22 23
  * month           (month) int64 1 2 3 4 5 6 7 8 9 10 11 12
  * year            (year) int64 1990 1991 1992 1993 ... 2017 2018 2019 2020
    XLONG           (south_north, west_east) float32 ...
    XLAT            (south_north, west_east) float32 ...
Dimensions without coordinates: south_north, west_east
Data variables:
    hourly_values   (hour, south_north, west_east) float32 ...
    monthly_values  (month, south_north, west_east) float32 ...
    yearly_values   (year, south_north, west_east) float32 ...
    overall_values  (south_north, west_east) float32 ...

In [2]:
root_dir = '/media/harish/SSD_4TB/EU_SCORES_project'
run = 'New_runs'

chunks={"Time": -1,"south_north": 8,"west_east": 8}
case = 'Portugal_coast'
file_name = 'T2'
variable = 'T2'
run_dir=f'{root_dir}/WRFV4.4/EU_SCORES/{run}/{case}/Postprocessed/variablewise_files'
target_dir=f'{root_dir}/WRFV4.4/EU_SCORES/{run}/{case}/Postprocessed/statistics_files/{file_name}'

ds = xr.open_dataset(f'{run_dir}/{file_name}.nc',chunks=chunks)[variable]
data = ds.isel(Time=slice(None,-1))
data

<xarray.DataArray 'T2' (Time: 271752, south_north: 127, west_east: 127)>
dask.array<getitem, shape=(271752, 127, 127), dtype=float32, chunksize=(271752, 8, 8), chunktype=numpy.ndarray>
Coordinates:
  * Time     (Time) datetime64[ns] 1990-01-01 ... 2020-12-31T23:00:00
Dimensions without coordinates: south_north, west_east
Attributes:
    FieldType:    104
    MemoryOrder:  XY 
    description:  TEMP at 2 M
    units:        K
    stagger:      
    coordinates:  XLONG XLAT XTIME

In [4]:
root_dir = '/media/harish/SSD_4TB/EU_SCORES_project'
run = 'New_runs'

chunks={"Time": -1,"south_north": 8,"west_east": 8}
case = 'Netherlands_coast'
file_name = 'T2'
variable = 'T2'
run_dir=f'{root_dir}/WRFV4.4/EU_SCORES/{run}/{case}/Postprocessed/variablewise_files'
target_dir=f'{root_dir}/WRFV4.4/EU_SCORES/{run}/{case}/Postprocessed/statistics_files/{file_name}'

ds = xr.open_dataset(f'{run_dir}/{file_name}.nc',chunks=chunks)[variable]
data = ds.isel(Time=slice(None,-1))

# chech if XLAT and XLONG are present in the dataset, if not, add them to data
if 'XLAT' not in data.coords or 'XLONG' not in data.coords:
    XLAND = xr.open_dataset(f'{run_dir}/XLAND.nc')
    XLAT = XLAND.XLAT
    XLONG = XLAND.XLONG
    data = data.assign_coords(XLAT=XLAT, XLONG=XLONG)

statistic = 'std'
with xr.open_dataset(f'{target_dir}/{statistic}.nc') as ds:
    ds['overall_values'] = overall_statistic(data,statistic)
    ds = ds.load()

# save the ds to the same file
ds.to_netcdf(f'{target_dir}/{statistic}.nc',mode='w')